In [1]:
import sys
sys.path.append('.')

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

In [3]:
# setting CUDA to load modules lazily
# Only available on CUDA 11.7+
# os.environ['CUDA_MODULE_LOADING'] = 'LAZY'

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader
from torchvision.io import read_image
from torchvision import transforms as T

In [4]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('runs/change_detection_exp_1')

2023-07-10 02:27:10.097757: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-10 02:27:10.246279: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-07-10 02:27:10.703525: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-07-10 02:27:10.703565: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [5]:
from change_detection.models.layers import layers
from change_detection.models import models
from change_detection import dataset
from loss.cross_entropy import EdgeCrossEntropy

In [6]:
from pynvml import *


def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()

In [7]:
print_gpu_utilization()

GPU memory occupied: 258 MB.


In [8]:
# # preparing cropped stuff for train, val, and test datasets
ds_path = Path("/home/m/data/projects/dr.eftekhari - change detection/CoTNet/dataset/LEVIR-CD").resolve()
# dataset.make_cropped_dataset(ds_path / "train")
# dataset.make_cropped_dataset(ds_path / "val")
# dataset.make_cropped_dataset(ds_path / "test")

In [9]:
train_ds = dataset.LevirCDDataset(ds_path, partition = "train")
val_ds = dataset.LevirCDDataset(ds_path, partition = "val")
test_ds = dataset.LevirCDDataset(ds_path, partition = "test")

In [10]:
len(train_ds), len(test_ds), len(val_ds)

(87220, 25088, 12544)

In [11]:
print_gpu_utilization()

GPU memory occupied: 258 MB.


In [12]:
# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

# Parameters
params = {'batch_size': 4,
          'shuffle': True,
          'num_workers': 6}
max_epochs = 2
learning_rate = 1e-3

# Dataloaders
train_loader = DataLoader(train_ds, **params) 
val_loader = DataLoader(val_ds, **params)
test_loader = DataLoader(test_ds, **params)

In [13]:
print_gpu_utilization()

GPU memory occupied: 258 MB.


In [14]:
# Define model
model = models.UnetCotnetNetwork(in_channels=3, classes=2, batch_size=params['batch_size'])
if not next(model.parameters()).is_cuda:
    model.to(device)
criterion = EdgeCrossEntropy()
# optimzer = optim.Adam(model.parameters(), lr=learning_rate)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [15]:
print_gpu_utilization()

GPU memory occupied: 1069 MB.


In [16]:
# Loop over epochs
for epoch in range(max_epochs):
    # Training
    for local_batch, local_labels in train_loader:
        # Transfer to GPU
        local_batch, local_labels = local_batch.to(device), local_labels.to(device)
        print(local_batch.shape)
        print(local_labels.shape)
        # Model computations
        out = model(local_batch[:, :3, :, :], local_batch[:, 3:, :, :])
        print(f"------ EPOCH: {epoch}-------")
        print_gpu_utilization()
        print(out.shape)
        local_batch.detach()
        local_labels.detach()
        print_gpu_utilization()
        torch.cuda.empty_cache()
        print_gpu_utilization()
        writer.add_scalar(np.random.randint())

#     # Validation
#     with torch.set_grad_enabled(False):
#         for local_batch, local_labels in validation_generator:
#             # Transfer to GPU
#             local_batch, local_labels = local_batch.to(device), local_labels.to(device)

#             # Model computations


torch.Size([4, 6, 128, 128])
torch.Size([4, 2, 128, 128])


/home/m/.local/lib/python3.10/site-packages/cupy/cuda/compiler.py:464: UserWarning: cupy.cuda.compile_with_cache has been deprecated in CuPy v10, and will be removed in the future. Use cupy.RawModule or cupy.RawKernel instead.
  warnings.warn(
/home/m/data/projects/dr.eftekhari - change detection/CoTNet/change_detection/models/models.py:46: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  out = torch.nn.functional.softmax(out)


idx: -4	conv12 --> torch.Size([4, 64, 128, 128])
idx: -3	conv14 --> torch.Size([4, 128, 64, 64])
idx: -2	conv16 --> torch.Size([4, 256, 32, 32])
idx: -1	conv18 --> torch.Size([4, 512, 8, 8])
idx: -4	conv68719476736 --> torch.Size([4, 64, 128, 128])
idx: -3	conv4398046511104 --> torch.Size([4, 128, 64, 64])
idx: -2	conv281474976710656 --> torch.Size([4, 256, 32, 32])
idx: -1	conv18014398509481984 --> torch.Size([4, 512, 8, 8])
dist.shape --> torch.Size([4, 512, 8, 8])

out shape from upsampling block -->  torch.Size([4, 512, 3, 3])


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 3 but got size 8 for tensor number 1 in the list.